In [75]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib2
import csv
import pandas
import os
from selenium.common.exceptions import NoSuchElementException



driver = webdriver.Firefox()
driver.get("http://www.hkexnews.hk/listedco/listconews/advancedsearch/search_active_main.aspx")

def get_release_date(stock_number):
    istock_code = driver.find_element_by_name("ctl00$txt_stock_code")
    istock_code.send_keys(stock_number)

    irelease_date = driver.find_element_by_xpath("//*[@id='ctl00_sel_DateOfReleaseFrom_y']/option[text()='1999']").click()

    ikeyword = driver.find_element_by_name("ctl00$txtKeyWord")
    ikeyword.send_keys("prospectus")

    submit = driver.find_element_by_xpath('//*[@id="aspnetForm"]/table/tbody/tr[7]/td[3]/label/a[1]/img').click()
    driver.implicitly_wait(5)
    assert "No results found." not in driver.page_source
    
    try:
        target_cell_text = driver.find_element_by_xpath('//*[@id="ctl00_gvMain"]/tbody/tr/td/a/u[text()=" Prospectus"]/../../../td[1]/span').text
        date = target_cell_text.split("\n",1)
        return date[0]
    except NoSuchElementException:
        return 'none'

def date_to_url(date, stock_number, language):
    stock_number = int(stock_number)
    date  = date.split("/",2)
    tdate = ''.join(date[::-1])
    base_url = 'http://www.hkexnews.hk/newlistings/prospectuses/'
    url = base_url + language + '_' + str(stock_number) + 'pro-' + tdate + '.pdf'
    return url
    
def read_csv():
    with open('Project28_HK IPO data.csv', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in spamreader:
            print ', '.join(row)

def main():
    df1 = pandas.DataFrame.from_csv('Project28_HK IPO data.csv', index_col=None)
    for i in range(len(df1)):
        stock_number = int(df1.iloc[i]['Stock Code'])
        release_date = get_release_date(stock_number)
        print release_date
        if(release_date != 'none'):
            url = date_to_url(release_date, stock_number, 'e')
            df1.loc[i, 'en'] = url
            download_file(url, stock_number)
        
        driver.find_element_by_xpath('//*[@id="aspnetForm"]/table[3]/tbody/tr[3]/td[1]/a/img').click()
        driver.close()
    print df1
    driver.quit()
    df1.to_csv("test.csv", index=False)
    
def download_file(download_url, stock_number):
    try:
        response = urllib2.urlopen(download_url)
        file = open("downloads/" + str(stock_number) + ".pdf", 'wb')
        file.write(response.read())
        file.close()
    except urllib2.HTTPError, e:
        pass


if __name__ == "__main__":
    main()



23/11/1999
25/01/2000
28/02/2000
05/04/2000
06/03/2000
27/03/2000
05/07/2002
none
17/01/2000
none
16/07/2001
none
30/11/2001
31/07/2001
05/12/1999
21/03/2000
none
18/11/1999
05/12/2000
none
none
30/03/2000
16/02/2000
none
01/06/2004
none
none
27/06/2000
19/04/2001
31/03/2003
24/10/2001
20/06/2000
28/09/2001
14/04/2000
09/06/2000
29/10/2001
10/04/2000
25/02/2002
27/07/2000
none
05/04/2000
10/05/2000
none
none
15/02/2001
none
20/07/2000
none
18/12/2001
none
none
04/09/2000
24/04/2002
07/07/2000
24/01/2002
22/12/2000
none
none
none
30/06/2003
15/07/2002
24/10/2003
09/10/2002
none
none
none
none
06/09/2005
29/11/2002
29/11/2002
none
none
31/10/2002
none
none
31/10/2003
none
none
16/02/2006
30/10/2003
08/06/2004
none
none
none
16/12/2002
none
23/06/2003
none
none
none
none
30/04/2003
none
none
none
04/08/2003
23/07/2003
none
09/02/2004
none
                                         Company Name  Stock Code    Year  \
0                Great World Company Holdings Limited        8003  1999.0  

NameError: name 'df1' is not defined